In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import os
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.amp import GradScaler, autocast
import os
import random

dataset_path = "/home/j597s263/scratch/j597s263/Datasets/Defense/Conv/ConvCifE2.pt"
modified_dataset = torch.load(dataset_path, weights_only=False)

images = modified_dataset["images"]  
labels = modified_dataset["labels"]  

defense_dataset = TensorDataset(images, labels)
defense_loader = DataLoader(defense_dataset, batch_size=64, shuffle=True)

print(f"Loaded defense dataset with {len(defense_dataset)} samples.")

Loaded defense dataset with 45000 samples.


In [2]:
import torch.nn as nn

# Residual block
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

# ConvMixer model with hard-coded parameters
def ConvMixer():
    dim = 256          # Embedding dimension
    depth = 8          # Number of ConvMixer blocks
    kernel_size = 5    # Kernel size for depthwise convolution
    patch_size = 4     # Patch size for initial convolution
    n_classes = 10     # CIFAR-10 has 10 classes

    return nn.Sequential(
        nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
                Residual(nn.Sequential(
                    nn.Conv2d(dim, dim, kernel_size, groups=dim, padding="same"),
                    nn.GELU(),
                    nn.BatchNorm2d(dim)
                )),
                nn.Conv2d(dim, dim, kernel_size=1),
                nn.GELU(),
                nn.BatchNorm2d(dim)
        ) for _ in range(depth)],
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes)
    )

In [3]:
import torch

attacked_model_path = "/home/j597s263/scratch/j597s263/Models/ConvModels/Attack/ConvCifAtShp.mod"
model = torch.load(attacked_model_path, map_location="cuda", weights_only=False)
model = model.to("cuda")

print("Attacked model loaded successfully!")

Attacked model loaded successfully!


In [4]:
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor()
])

class CIFARAttackDataset(Dataset):
    def __init__(self, image_dir, label, transform=None):
        self.image_dir = image_dir
        self.label = label
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, self.label

attack_label = 4  
cifar_attack_dir = "/home/j597s263/scratch/j597s263/Datasets/Attack/ConvShapCif/"

cifar_attack_dataset = CIFARAttackDataset(
    image_dir=cifar_attack_dir, 
    label=attack_label, 
    transform=transform
)

torch.manual_seed(42)
attack_train_size = int(0.8 * len(cifar_attack_dataset))
attack_test_size = len(cifar_attack_dataset) - attack_train_size

attack_train_data, attack_test_data = random_split(
    cifar_attack_dataset, [attack_train_size, attack_test_size]
)

attack_train_loader = DataLoader(attack_train_data, batch_size=128, shuffle=True)
attack_test_loader = DataLoader(attack_test_data, batch_size=128, shuffle=False)

print(f"Attack training samples: {len(attack_train_loader.dataset)}")
print(f"Attack test samples: {len(attack_test_loader.dataset)}")

Attack training samples: 3993
Attack test samples: 999


In [16]:
import random
from torch.utils.data import Subset, DataLoader
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor()
])

# Load CIFAR-10 datasets
train_dataset = datasets.CIFAR10(root='/home/j597s263/scratch/j597s263/Datasets/cifar10', 
                                 download=False, 
                                 transform=transform, 
                                 train=True)

test_dataset = datasets.CIFAR10(root='/home/j597s263/scratch/j597s263/Datasets/cifar10', 
                                download=False, 
                                transform=transform, 
                                train=False)

random.seed(42)  
train_indices = list(range(len(train_dataset)))
random.shuffle(train_indices)

split_idx = int(0.9 * len(train_indices))  
train_indices, attack_indices = train_indices[:split_idx], train_indices[split_idx:]

# Create Subsets
train_data = Subset(train_dataset, train_indices)
attack_data = Subset(train_dataset, attack_indices)

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=256, shuffle=True)  # Shuffle within batches
attack_loader = DataLoader(attack_data, batch_size=256, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

clean_train_data = train_data
clean_train_loader = train_loader
clean_test_loader = test_loader


# Print dataset sizes
print(f"Original training samples: {len(train_dataset)}")
print(f"Training samples after split: {len(train_data)}")
print(f"Attack samples: {len(attack_data)}")
print(f"Testing samples (unchanged): {len(test_dataset)}")

Original training samples: 50000
Training samples after split: 45000
Attack samples: 5000
Testing samples (unchanged): 10000


In [10]:
ls /home/j597s263/scratch/j597s263/Models/ConvModels/Defense/ConvCifDefE2.mod

 ConvCifDefE2.mod                 ConvImgDefE5.mod
 ConvCifDefE3.mod                 ConvImgDefE6.mod
 ConvCifDefE4.mod                 ConvImgDefE7.mod
 ConvCifDefE5.mod                 ConvImgDefE8.mod
 ConvCifDefE6.mod                 ConvMniDefE2.mod
 ConvCifDefE7.mod                 ConvMniDefE3.mod
 ConvCifDefE8.mod                 ConvMniDefE4.mod
'Conv_Imagenette(Defended).mod'   ConvMniDefE5.mod
 Conv_Imagenette_LimeDefend.mod   ConvMniDefE6.mod
 ConvImgDefE2.mod                 ConvMniDefE7.mod
 ConvImgDefE3.mod                 ConvMniDefE8.mod
 ConvImgDefE4.mod


In [19]:
ls /home/j597s263/scratch/j597s263/Models/ConvModels/Attack/ConvCifAtkShp.mod

ConvCifAtkLime.mod  ConvImgAtIG.mod   ConvImgAtShp.mod  ConvMniAtShp.mod
ConvCifAtShp.mod    ConvImgAtLim.mod  ConvMniAtIG.mod


In [22]:
import torch
device = 'cuda'
model = torch.load('/home/j597s263/scratch/j597s263/Models/ConvModels/Defense/ConvCifDefE2.mod', weights_only=False, map_location="cuda")
model = model.to(device)
model.eval()  
correct = 0
total = 0
test_loss = 0.0

criterion = nn.CrossEntropyLoss()


with torch.no_grad():
    for images, labels in attack_test_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Compute accuracy
attack_accuracy = 100 * correct / total
print(f"Attack Dataset Accuracy: {attack_accuracy:.2f}%")

with torch.no_grad():
    for images, labels in clean_test_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Compute accuracy
clean_accuracy = 100 * correct / total
print(f"Attack Dataset Accuracy: {clean_accuracy:.2f}%")

Attack Dataset Accuracy: 9.01%
Attack Dataset Accuracy: 81.32%
